In [ ]:
cd ..

In [ ]:
# load packages
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa
import matplotlib.pyplot as plt
from utils.ASR.models import ASRCNN
from utils.JDC.model import JDCNet
from models import Generator, MappingNetwork, StyleEncoder
import soundfile as sf
import IPython.display as ipd
import pyworld
from tqdm import tqdm
%matplotlib inline

In [ ]:
speakers = ['F101', 'F102', 'F103', 'F104', 'F105', 'F106', 'F107', 'F108', 'F109', 'F110',
            'M101', 'M102', 'M103', 'M104', 'M105', 'M106', 'M107', 'M108', 'M109', 'M110',
            'FAF', 'FFS', 'FKM', 'FKN', 'FKS', 'FMS', 'FSU', 'FTK', 'FYM', 'FYN',
            'MAU', 'MHT', 'MMS', 'MMY', 'MNM', 'MSH', 'MTK', 'MTM', 'MTT', 'MXM']
print(len(speakers))
to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def build_model(model_params={}):
    args = Munch(model_params)
    generator = Generator(args.dim_in, args.style_dim, args.max_conv_dim, w_hpf=args.w_hpf, F0_channel=args.F0_channel)
    mapping_network = MappingNetwork(args.latent_dim, args.style_dim, args.num_domains, hidden_dim=args.max_conv_dim)
    style_encoder = StyleEncoder(args.dim_in, args.style_dim, args.num_domains, args.max_conv_dim)

    nets_ema = Munch(generator=generator,
                     mapping_network=mapping_network,
                     style_encoder=style_encoder)

    return nets_ema

def compute_style(model, speaker_dicts):
    reference_embeddings = {}
    for key, (path, speaker) in speaker_dicts.items():
        if path == "":
            label = torch.LongTensor([speaker]).to('cuda')
            latent_dim = model.mapping_network.shared[0].in_features
            ref = model.mapping_network(torch.randn(1, latent_dim).to('cuda'), label)
        else:
            print(path)
            wave, sr = librosa.load(path, sr=24000)
            audio, index = librosa.effects.trim(wave, top_db=30)
            if sr != 24000:
                wave = librosa.resample(wave, sr, 24000)
            mel_tensor = preprocess(wave).to('cuda')

            with torch.no_grad():
                label = torch.LongTensor([speaker])
                ref = model.style_encoder(mel_tensor.unsqueeze(1), label)
        reference_embeddings[key] = (ref, label)

    return reference_embeddings

In [ ]:
# load F0 model

F0_model = JDCNet(num_class=1, seq_len=192)
params = torch.load("utils/JDC/bst.t7")['net']
F0_model.load_state_dict(params)
_ = F0_model.eval()
F0_model = F0_model.to('cuda')

In [ ]:
# load vocoder
from parallel_wavegan.utils import load_model
vocoder = load_model("Vocoder/checkpoint-400000steps.pkl").to('cuda').eval()
vocoder.remove_weight_norm()
_ = vocoder.eval()

In [ ]:
# load starganv2

model_path = 'Models/atr/epoch_00032.pth'

with open('Configs/config.yml') as f:
    starganv2_config = yaml.safe_load(f)
starganv2 = build_model(model_params=starganv2_config["model_params"])
params = torch.load(model_path, map_location='cpu')
print("Epochs:", params["epochs"])

params = params['model_ema']
_ = [starganv2[key].load_state_dict(params[key]) for key in starganv2]
_ = [starganv2[key].eval() for key in starganv2]
starganv2.style_encoder = starganv2.style_encoder.to('cuda')
starganv2.mapping_network = starganv2.mapping_network.to('cuda')
starganv2.generator = starganv2.generator.to('cuda')


## Conversion

In [ ]:
def scale_db(y, target_dB_FS=-25, eps=1e-6):
    rms = np.sqrt(np.mean(y ** 2))
    scalar = 10 ** (target_dB_FS / 20) / (rms + eps)
    y *= scalar
    return y

In [ ]:
# no reference, using mapping network
speaker_dicts = {}
for s in speakers:
    speaker_dicts[s] = ("data/ATR_processed/wav24/%s/1.wav" % s,
                        speakers.index(s))
reference_embeddings = compute_style(starganv2, speaker_dicts)

In [ ]:
# conversion
import time
start = time.time()
wav_path = "data/samples/M105/M105SF_A01.AD.wav"
audio, source_sr = librosa.load(wav_path, sr=24000)
print(audio.shape)
audio = audio / np.max(np.abs(audio))
ipd.display(ipd.Audio(audio, rate=24000))
source = preprocess(audio).to('cuda:0')
keys = []
converted_samples = {}
reconstructed_samples = {}
converted_mels = {}
style_pred = {}
for key, (ref, _) in reference_embeddings.items():
    with torch.no_grad():
        f0_feat = F0_model.get_feature_GAN(source.unsqueeze(1))
        out = starganv2.generator(source.unsqueeze(1), ref, F0=f0_feat)
        c = out.transpose(-1, -2).squeeze().to('cuda')
        y_out = vocoder.inference(c)
        y_out = y_out.view(-1).cpu()
        recon = None
        if key not in speaker_dicts or speaker_dicts[key][0] == "":
            recon = None
        else:
            wave, sr = librosa.load(speaker_dicts[key][0], sr=24000)
            mel = preprocess(wave)
            c = mel.transpose(-1, -2).squeeze().to('cuda')
            recon = vocoder.inference(c)
            recon = recon.view(-1).cpu().numpy()

    converted_samples[key] = y_out.numpy()
    reconstructed_samples[key] = recon

    converted_mels[key] = out

    keys.append(key)
end = time.time()
print('total processing time: %.3f sec' % (end - start) )


for key, wave in converted_samples.items():
    print('Converted: %s' % key)
    ipd.display(ipd.Audio(wave, rate=24000))
    print('Reference (vocoder): %s' % key)
    if reconstructed_samples[key] is not None:
        ipd.display(ipd.Audio(reconstructed_samples[key], rate=24000))

print('Original (vocoder):')
wave, sr = librosa.load(wav_path, sr=24000)
mel = preprocess(wave)
c = mel.transpose(-1, -2).squeeze().to('cuda')
with torch.no_grad():
    recon = vocoder.inference(c)
    recon = recon.view(-1).cpu().numpy()
ipd.display(ipd.Audio(recon, rate=24000))
print('Original:')
ipd.display(ipd.Audio(wav_path, rate=24000))